In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from PIL import Image
from io import BytesIO

### WEB SCRAPING

In [2]:
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
url = 'https://www.investing.com/equities/nike'
response = requests.get(url,headers=headers)

In [3]:
print(response.status_code)

200


In [4]:
soup = BeautifulSoup(response.content, 'lxml')
soup

<!DOCTYPE html>
<html class="html" dir="ltr" lang="en"><head><link href="//i-invdn-com.investing.com" rel="dns-prefetch"/><link href="//securepubads.g.doubleclick.net" rel="dns-prefetch"/><link href="//fonts.googleapis.com" rel="dns-prefetch"/><link href="//i-invdn-com.investing.com" rel="preconnect"/><link href="//securepubads.g.doubleclick.net" rel="preconnect"/><link href="//live.primis.tech" rel="preconnect"/><link href="//fonts.googleapis.com" rel="preconnect"/><script>(function(){var __pm_s_v="83c155b";var pmJsonpEndPoint="CURRENT_JSONP_ENDPOINT";var pmJSHost="https://js.ad-score.com";var pid="1000745";var predefinedTags={tid:{qc:"utm_source",m:"refhost",v:""},l1:{qc:"utm_subsource",v:""},l2:{qc:"utm_medium",v:""},l3:{qc:"utm_campaign",v:""},l4:{qc:"utm_content",v:""},l5:window.location.hostname,l6:window.location.pathname};var runOnInnerPages=true;var predefinedTimeout=1500;var predefinedRequestsBeforeJSONP=3;var predefinedSampleRateRT=1;var predefinedSampleRateJS=
1;var predefi

In [5]:
soup.find('title')

<title>Nike Stock Price Today | NYSE NKE Live Ticker - Investing.com</title>

In [6]:
soup.find_all('span', class_='sc-1ow4cwt-1 ieFnWP')

[]

In [56]:
company = soup.find('h1', {'class': 'text-2xl font-semibold instrument-header_title__GTWDv mobile:mb-2'}).text
price = soup.find('div', {'class': 'instrument-price_instrument-price__3uw25 flex items-end flex-wrap font-bold'}).find_all('span')[0].text
change = soup.find('div', {'class': 'instrument-price_instrument-price__3uw25 flex items-end flex-wrap font-bold'}).find_all('span')[2].text
currency = soup.find('div', {'class': "instrument-metadata_currency__3RQjH"}).find_all('span')[1].text
time_stamp = soup.find('div', {'class': 'instrument-metadata_time__3kNQ2'}).find('time').text
market = soup.find('div', {'class': "relative-selector_relative-selector__1ZxRl"}).find('span').text
volume = soup.find('div', {'class': 'trading-hours_value__2MrOn'}).text
market_cap = (soup.find('dd', {'data-test': 'marketCap'}).find('span', {'class': 'key-info_dd-numeric__2cYjc'}).find('span').text) + (soup.find('dd', {'data-test': 'marketCap'}).find('span', {'class': 'key-info_dd-numeric__2cYjc'}).find_all('span')[1].text)



In [57]:
print('Loading: ', url)
print(company, price, change, currency, time_stamp, market, volume, market_cap)

Loading:  https://www.investing.com/equities/dangcem
Dangote Cement PLC (DANGCEM) 243.00 (-6.11%) NGN 09:30:00 Lagos 465,912 4.37T


In [62]:
urls = [
    'https://www.investing.com/equities/nike',
    'https://www.investing.com/equities/coca-cola-co',
    'https://www.investing.com/equities/microsoft-corp',
    'https://www.investing.com/equities/amazon-com-inc',
    'https://www.investing.com/equities/goldman-sachs-group',
    'https://www.investing.com/equities/tesla-motors',
    'https://www.investing.com/equities/netflix,-inc.',
    'https://www.investing.com/equities/facebook-inc',
    'https://www.investing.com/equities/alibaba',
    'https://www.investing.com/equities/dangcem',
    'https://www.investing.com/equities/axa',
    'https://www.investing.com/equities/hellofresh-se',
    'https://www.investing.com/equities/lloyds-banking-grp',
    'https://www.investing.com/equities/sap-ag',
    'https://www.investing.com/equities/eads',
        
]

In [63]:
df_list = []
  
for url in urls:
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
  
    company = soup.find('h1', {'class': 'text-2xl font-semibold instrument-header_title__GTWDv mobile:mb-2'}).text
    price = soup.find('div', {'class': 'instrument-price_instrument-price__3uw25 flex items-end flex-wrap font-bold'}).find_all('span')[0].text
    change = soup.find('div', {'class': 'instrument-price_instrument-price__3uw25 flex items-end flex-wrap font-bold'}).find_all('span')[2].text.strip("()")[:-1]
    currency = soup.find('div', {'class': "instrument-metadata_currency__3RQjH"}).find_all('span')[1].text
    time_stamp = soup.find('div', {'class': 'instrument-metadata_time__3kNQ2'}).find('time').text
    market = soup.find('div', {'class': "relative-selector_relative-selector__1ZxRl"}).find('span').text
    volume = soup.find('div', {'class': 'trading-hours_value__2MrOn'}).text
    market_cap = (soup.find('dd', {'data-test': 'marketCap'}).find('span', {'class': 'key-info_dd-numeric__2cYjc'}).find('span').text) + (soup.find('dd', {'data-test': 'marketCap'}).find('span', {'class': 'key-info_dd-numeric__2cYjc'}).find_all('span')[1].text)

    
    df_list.append({'company': company,
                    'price': float(price),
                    'change_%': float(change),
                    'currency': currency,
                    'market': market,
                    'volume': volume,
                    'market_cap': market_cap,
                    'time_stamp': time_stamp,
                    })
      
    #print('Loading: ', url)
    #print(company, price, change)
    #print(df_list)

In [65]:
df = pd.DataFrame(df_list)
df.head(20)

,company,price,change_%,currency,market,volume,market_cap,time_stamp
0,Nike Inc (NKE),116.08,-0.82,USD,NYSE,"2,147,948",181.86B,12:53:29
1,Coca-Cola Co (KO),64.97,0.15,USD,NYSE,"2,256,818",280.88B,12:57:38
2,Microsoft Corporation (MSFT),290.64,-0.23,USD,NASDAQ,"9,197,650",2.16T,12:58:39
3,Amazon.com Inc (AMZN),142.21,0.07,USD,NASDAQ,"20,274,566",1.45T,12:58:35
4,Goldman Sachs Group Inc (GS),352.00,-0.49,USD,NYSE,"451,216",114.5B,12:58:43
5,Tesla Inc (TSLA),910.31,-0.18,USD,NASDAQ,"10,677,041",952.14B,12:58:22
6,Netflix Inc (NFLX),244.60,1.43,USD,NASDAQ,"2,890,169",109.02B,12:58:18
7,Meta Platforms Inc (META),174.51,-0.19,USD,NASDAQ,"11,639,877",468.04B,12:58:08
8,Alibaba Group Holdings Ltd ADR (BABA),90.39,0.69,USD,NYSE,"9,876,575",237.67B,12:58:10
9,Dangote Cement PLC (DANGCEM),243.00,-6.11,NGN,Lagos,"465,912",4.37T,09:30:00
